In [64]:

import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import re
import csv
from time import sleep
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv()

False

In [65]:
df = pd.read_excel(f"D:\Study\Level4\grad project\linkedin scraping trial\scraping_result4.xlsx")

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1971 entries, 0 to 1970
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         1971 non-null   int64  
 1   co-founder1                   1971 non-null   object 
 2   co-founder2                   1022 non-null   object 
 3   organization_name             1971 non-null   object 
 4   country                       1964 non-null   object 
 5   google_search_query_founder1  1971 non-null   object 
 6   google_search_query_founder2  1022 non-null   object 
 7   co-founder1 linkedin profile  1604 non-null   object 
 8   co-founder2 linkedin profile  5 non-null      object 
 9   co-founder1 cv                1604 non-null   object 
 10  co-founder2 cv                0 non-null      float64
 11  notes                         82 non-null     object 
 12  key                           1971 non-null   object 
dtypes: 

In [38]:
#EMAILS = ['mostafaeldaly123456789@gmail.com','yousefeldaly23@gmail.com', 'yousefeldaly93@gmail.com', '20220527@stud.fci-cu.edu.eg',
#        '18102@stemegypt.edu.eg', 'mohamedbadr861686@gmail.com']
EMAILS = ['yousefeldaly23@gmail.com']
email = "lipiha6905@2mik.com"
PASSWORD = '#0141922425HHeeyy'

In [39]:

driver = webdriver.Chrome()
#driver.get('https://www.linkedin.com/login')
print(driver.title)

# local_html_path = r"C:\Users\Yusuf Aldaly\Downloads\practical\drive-download-20250517T063722Z-1-001\Smart-food\Smart-food\home.html"
# file_url = 'file:///' + os.path.abspath(local_html_path).replace("\\", "/")

# options = webdriver.ChromeOptions()

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# try:
#     driver.get(file_url)
#     print("[✓] Opened Home Page")

#     egyptian_link = driver.find_element(By.LINK_TEXT, "Egyptian")
#     egyptian_link.click()
#     time.sleep(1)
#     print("[✓] Navigated to Egyptian Dishes Page")

# finally:
driver.quit()


In [40]:
def login(driver, user_email, user_password):
    email = driver.find_element(By.ID, 'username')
    email.send_keys(user_email)

    password = driver.find_element(By.ID, 'password')
    password.send_keys(user_password)

    password.submit()

In [41]:
def is_logged_in(driver, timeout=10):
    try:
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.ID, "ember35"))  # Search bar in the top navbar
        )
        return True
    except:
        return False

In [42]:
def clean_repeated_phrases(text):
    """
    Removes immediate word or phrase repetitions within a single string.
    Example: "Founder Founder" -> "Founder"
    "Cairo Cairo" -> "Cairo"
    "Issued Jun 2010 Issued Jun 2010" -> "Issued Jun 2010"
    """
    words = text.split()
    cleaned = []
    prev = None
    for word in words:
        if word != prev:
            cleaned.append(word)
        prev = word
    return ' '.join(cleaned)

def extract_info_from_items(items):
    seen = set()
    unique_cleaned_items = []

    for item in items:
        text = item.get_text(separator=' ', strip=True)
        cleaned_text = clean_repeated_phrases(text)
        if cleaned_text not in seen:
            seen.add(cleaned_text)
            unique_cleaned_items.append(cleaned_text)

    return '\n'.join(unique_cleaned_items)


In [43]:
def extract_headline(headline_div):
    if headline_div:
        return headline_div.get_text(separator=' ', strip=True)
    else:
        return "No headline found"

In [44]:
def extract_profile_summary(about_section):
    if not about_section:
        return "No about info"
    raw_text = about_section.get_text(separator=' ', strip=True)
    cleaned_text = clean_repeated_phrases(raw_text)
    return cleaned_text


In [45]:
def extract_info_from_section_items(section):
    items = section.find_all('li', class_='artdeco-list__item')
    return extract_info_from_items(items)

In [46]:
def extract_info_from_page_section_items(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    section = soup.find('section', class_=['artdeco-card', 'pb3'])
    items = section.find_all('li', class_='pvs-list__paged-list-item')
    
    info = extract_info_from_items(items)

    back_button = driver.find_element(By.XPATH, "//button[@aria-label='Back to the main profile page']")
    back_button.click()

    return info

In [47]:
def extract_experience(driver, section):
    co_founder_experience = ""
    try:
        button = driver.find_element(By.ID, "navigation-index-see-all-experiences")
        button.click()
        sleep(3)

        co_founder_experience = extract_info_from_page_section_items(driver)

    except NoSuchElementException:
        co_founder_experience = extract_info_from_section_items(section)
    
    return co_founder_experience or "No experience info found"


In [48]:
def extract_education(driver, section):
    co_founder_education = ""
    try:
        button = driver.find_element(By.ID, "navigation-index-see-all-education")
        button.click()
        sleep(3)

        co_founder_education = extract_info_from_page_section_items(driver)
    except NoSuchElementException:
        co_founder_education = extract_info_from_section_items(section)
    return co_founder_education or "No education info found"

In [49]:
def extract_licenses_and_certifications(driver, section):
    co_founder_licenses_and_certifications = ""
    try:
        button = driver.find_element(By.ID, "navigation-index-see-all-licenses-and-certifications")
        button.click()
        sleep(3)

        co_founder_licenses_and_certifications = extract_info_from_page_section_items(driver)
    except NoSuchElementException:
        co_founder_licenses_and_certifications = extract_info_from_section_items(section)

    return co_founder_licenses_and_certifications or "No licenses or certifications info found"

In [50]:
def extract_skills(driver, section):
    sleep(3)
    co_founder_skills = ""

    span_element = section.find("span", class_="pvs-navigation__text")

    if span_element is not None:
        try:
            span_text = span_element.text.strip().replace(' ', '-')
            button = driver.find_element(By.ID, f"navigation-index-{span_text}")
            button.click()
            sleep(3)

            co_founder_skills = extract_info_from_page_section_items(driver)

        except NoSuchElementException:
            co_founder_skills = extract_info_from_section_items(section)

    else:
        co_founder_skills = extract_info_from_section_items(section)

    return co_founder_skills or "No skills info found"

In [51]:
# def extract_skills(driver, section):
#     co_founder_skills = ""
#     try:
#         sleep(3)
#         span_element = section.find("span", class_="pvs-navigation__text")
#         span_text = span_element.text.strip().replace(' ', '-')
#         button = driver.find_element(By.ID, f"navigation-index-{span_text}")
#         button.click()
#         sleep(4)

#         co_founder_skills = extract_info_from_page_section_items(driver)
#     except NoSuchElementException:
#         co_founder_skills = extract_info_from_section_items(section)

#     return co_founder_skills or "No skills info found"


In [52]:
# def extract_skills(driver, section):
#     co_founder_skills = ""
#     try:
#         sleep(3)
#         span_element = section.find("span", class_="pvs-navigation__text")
#         span_text = span_element.text.strip().replace(' ', '-')
#         button_elements = driver.find_elements(By.ID, f"navigation-index-{span_text}")
        
#         if button_elements:
#             button_elements[0].click()
#             sleep(4)
#             co_founder_skills = extract_info_from_page_section_items(driver)
#         else:
#             co_founder_skills = extract_info_from_section_items(section)

#     except NoSuchElementException:
#         co_founder_skills = extract_info_from_section_items(section)

#     return co_founder_skills or "No skills info found"


In [53]:
def extract_cofounder_info(driver, url):
    
    sleep(3)
    co_founder_info = ""
    driver.get(url)
    sleep(7)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    sections = soup.find_all('section', {'class': 'artdeco-card pv-profile-card break-words mt2'})


    headline_div = soup.find('div', {'class': 'text-body-medium break-words'})
    if headline_div:
        headline = extract_headline(headline_div)
        co_founder_info += f"Headline:\n{headline}\n\n"

    for sec in sections:
            sleep(5)
            if sec.find('div', {'id': 'about'}):
                co_founder_info += f"\n\nAbout:\n" + extract_profile_summary(sec)
                print("About section found and extracted.")
            elif sec.find('div', {'id': 'experience'}):
                co_founder_info += "\n\nExperience:\n" + extract_experience(driver, sec)
                print("Experience section found and extracted.")
            elif sec.find('div', {'id': 'education'}):
                co_founder_info += "\n\nEducation:\n" + extract_education(driver, sec)
                print("Education section found and extracted.")
            elif sec.find('div', {'id': 'licenses_and_certifications'}):
                co_founder_info += "\n\nLicences and Certifications:\n" + extract_licenses_and_certifications(driver, sec)
                print("Licenses and Certifications section found and extracted.")
            elif sec.find('div', {'id': 'skills'}):
                co_founder_info += "\n\nSkills\n" + extract_skills(driver, sec)
                print("Skills section found and extracted.")


    # driver.quit()
    if co_founder_info:
        return co_founder_info
    else:
        return None

In [54]:
processed_counter = {'count': 0}  

def process_row(row):
    try:
        url = row['co-founder1 linkedin profile']
        if pd.notna(url) and pd.isna(row['co-founder1 cv']):
            processed_counter['count'] += 1
            print(f"[{processed_counter['count']}] extracting info of co-founder {row['co-founder1']}\n")
            sleep(15)
            info = extract_cofounder_info(driver, url)
            if info is None:
                raise ValueError("Login failed or data not found")
            return info
    except Exception as e:
        print(f"⚠️ Error processing {row['co-founder1']}: {e}")
    return None


In [55]:
df1 = df.head(1972).copy()

In [56]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1971 entries, 0 to 1970
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         1971 non-null   int64  
 1   co-founder1                   1971 non-null   object 
 2   co-founder2                   1022 non-null   object 
 3   organization_name             1971 non-null   object 
 4   country                       1964 non-null   object 
 5   google_search_query_founder1  1971 non-null   object 
 6   google_search_query_founder2  1022 non-null   object 
 7   co-founder1 linkedin profile  1604 non-null   object 
 8   co-founder2 linkedin profile  5 non-null      object 
 9   co-founder1 cv                1583 non-null   object 
 10  co-founder2 cv                0 non-null      float64
 11  notes                         82 non-null     object 
 12  key                           1971 non-null   object 
dtypes: 

In [57]:
df1.head()

,index,co-founder1,co-founder2,organization_name,country,google_search_query_founder1,google_search_query_founder2,co-founder1 linkedin profile,co-founder2 linkedin profile,co-founder1 cv,co-founder2 cv,notes,key
0,0,Ahmed Aboulella,NaN,InfiniLink,Egypt,"site:linkedin.com/in Ahmed Aboulella ""InfiniLi...",NaN,https://eg.linkedin.com/in/ahmedfaboulella,NaN,Headline:\nCo-Founder & CEO @ InfiniLink\n\n\n...,NaN,NaN,"('Ahmed Aboulella', 'InfiniLink')"
1,1,Ahmed Sabbah,Youssef Sholqamy,Telda,Egypt,"site:linkedin.com/in Ahmed Sabbah ""Telda"" Egypt","site:linkedin.com/in Youssef Sholqamy ""Telda"" ...",https://eg.linkedin.com/in/ahmedsabbah,NaN,Headline:\nCEO at Telda - We're hiring!\n\n\n\...,NaN,NaN,"('Ahmed Sabbah', 'Telda')"
2,2,Hamza Tag,Yosra Badr,Grinta,Egypt,"site:linkedin.com/in Hamza Tag ""Grinta"" Egypt","site:linkedin.com/in Yosra Badr ""Grinta"" Egypt",https://eg.linkedin.com/in/hamza-tag-a8751579,NaN,Headline:\nCo-founder @ Grinta\n\n\n\nAbout:\n...,NaN,NaN,"('Hamza Tag', 'Grinta')"
3,3,Waleed Sadek,NaN,PaySky,Egypt,"site:linkedin.com/in Waleed Sadek ""PaySky"" Egypt",NaN,https://eg.linkedin.com/in/waleed-sadek,NaN,Headline:\nFounder & CEO at PaySky | Yalla Sup...,NaN,NaN,"('Waleed Sadek', 'PaySky')"
4,4,Ayman Essawy,NaN,Connect Money,Egypt,"site:linkedin.com/in Ayman Essawy ""Connect Mon...",NaN,https://eg.linkedin.com/in/aymanessawy,NaN,Headline:\nTech entrepreneur | Connect money |...,NaN,NaN,"('Ayman Essawy', 'Connect Money')"


In [58]:
driver = webdriver.Chrome()
sleep(1)
driver.get('https://www.linkedin.com/login')
sleep(2)
password = PASSWORD

login(driver, email, password)
sleep(30)
if is_logged_in(driver):
    print(f"logged in with email{email}")
    df1['co-founder1 cv'] = df1.apply(
        lambda row: process_row(row), axis=1
    )
    print(f"✅ Finished processing with email: {email}")
else:
    print("Can not log in with this email")

logged in with emaillipiha6905@2mik.com
[1] extracting info of co-founder Osama ElEnany

About section found and extracted.
Experience section found and extracted.
Education section found and extracted.
Skills section found and extracted.
[2] extracting info of co-founder Arun Ramachandran

About section found and extracted.
Experience section found and extracted.
Education section found and extracted.
Licenses and Certifications section found and extracted.
Skills section found and extracted.
[3] extracting info of co-founder Edmond Husseini

About section found and extracted.
Experience section found and extracted.
Education section found and extracted.
Skills section found and extracted.
[4] extracting info of co-founder Saygin Yalcin

About section found and extracted.
Experience section found and extracted.
Education section found and extracted.
Skills section found and extracted.
[5] extracting info of co-founder Ahmed Ismail

Experience section found and extracted.
Education sec

In [59]:
#A. Mohammed
#Zeeshan Sajid
#Leo David


In [60]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1971 entries, 0 to 1970
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         1971 non-null   int64  
 1   co-founder1                   1971 non-null   object 
 2   co-founder2                   1022 non-null   object 
 3   organization_name             1971 non-null   object 
 4   country                       1964 non-null   object 
 5   google_search_query_founder1  1971 non-null   object 
 6   google_search_query_founder2  1022 non-null   object 
 7   co-founder1 linkedin profile  1604 non-null   object 
 8   co-founder2 linkedin profile  5 non-null      object 
 9   co-founder1 cv                21 non-null     object 
 10  co-founder2 cv                0 non-null      float64
 11  notes                         82 non-null     object 
 12  key                           1971 non-null   object 
dtypes: 

In [61]:
cv_map = df1.set_index('key')['co-founder1 cv']

df['co-founder1 cv'] = df['key'].map(cv_map).combine_first(df['co-founder1 cv'])

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1971 entries, 0 to 1970
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         1971 non-null   int64  
 1   co-founder1                   1971 non-null   object 
 2   co-founder2                   1022 non-null   object 
 3   organization_name             1971 non-null   object 
 4   country                       1964 non-null   object 
 5   google_search_query_founder1  1971 non-null   object 
 6   google_search_query_founder2  1022 non-null   object 
 7   co-founder1 linkedin profile  1604 non-null   object 
 8   co-founder2 linkedin profile  5 non-null      object 
 9   co-founder1 cv                1604 non-null   object 
 10  co-founder2 cv                0 non-null      float64
 11  notes                         82 non-null     object 
 12  key                           1971 non-null   object 
dtypes: 

In [63]:
df.to_excel("scraping_result4.xlsx", index=False)